<a href="https://colab.research.google.com/github/hughmck/COMP472-Assignment1/blob/main/472A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comp 472 Assignment 1


## Process
Step 1: Preprocessing
Step 2 : 

### Step 1 : Preprocessing

In [27]:
import pandas as pd
import numpy as np
import json


In [35]:
df = pd.read_json('goemotions.json')
df

,0,1,2
0,That game hurt.,sadness,negative
1,"You do right, if you don't care then fuck 'em!",neutral,neutral
2,Man I love reddit.,love,positive
3,"[NAME] was nowhere near them, he was by the Fa...",neutral,neutral
4,Right? Considering it’s such an important docu...,gratitude,positive
...,...,...,...
171815,"Well, I'm glad you're out of all that now. How...",joy,positive
171816,Everyone likes [NAME].,love,positive
171817,Well when you’ve imported about a gazillion of...,caring,positive
171818,That looks amazing,admiration,positive


#test